In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [4]:
listings = "listings.csv"
listings_data_df = pd.read_csv(listings)
listings_data_df.tail()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
22547,29856708,Cozy Apartment right in the center of Berlin,87555909,Ulisses,Mitte,Brunnenstr. Süd,52.533865,13.400731,Entire home/apt,60,2,0,NaN,NaN,1,314
22548,29857108,Altbau/ Schöneberger Kiez / Schlafsofa,67537363,Jörg,Tempelhof - Schöneberg,Schöneberg-Nord,52.496211,13.341738,Shared room,20,1,0,NaN,NaN,6,78
22549,29864272,Artists loft with garden in the center of Berlin,3146923,Martin,Pankow,Prenzlauer Berg Südwest,52.531800,13.411999,Entire home/apt,85,3,0,NaN,NaN,2,15
22550,29866805,Room for two with private shower / WC,36961901,Arte Luise,Mitte,Alexanderplatz,52.520802,13.378688,Private room,99,1,0,NaN,NaN,3,6
22551,29867352,"Sunny, modern and cozy flat in Berlin Neukölln :)",177464875,Sebastian,Neukölln,Schillerpromenade,52.473762,13.424447,Private room,45,5,0,NaN,NaN,1,21


In [4]:
new_listings_data_df = listings_data_df[['id', 'name', 'neighbourhood', 'price']].copy()
new_listings_data_df.head()

,id,name,neighbourhood,price
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,Brunnenstr. Süd,60
1,2695,Prenzlauer Berg close to Mauerpark,Prenzlauer Berg Nordwest,17
2,3176,Fabulous Flat in great Location,Prenzlauer Berg Südwest,90
3,3309,BerlinSpot Schöneberg near KaDeWe,Schöneberg-Nord,26
4,7071,BrightRoom with sunny greenview!,Helmholtzplatz,42


In [5]:
reviews = "reviews_summary.csv"
reviews_data_df = pd.read_csv(reviews)
reviews_data_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2015,69544350,2016-04-11,7178145,Rahel,Mein Freund und ich hatten gute gemütliche vie...
1,2015,69990732,2016-04-15,41944715,Hannah,Jan was very friendly and welcoming host! The ...
2,2015,71605267,2016-04-26,30048708,Victor,Un appartement tres bien situé dans un quartie...
3,2015,73819566,2016-05-10,63697857,Judy,"It is really nice area, food, park, transport ..."
4,2015,74293504,2016-05-14,10414887,Romina,"Buena ubicación, el departamento no está orden..."


In [6]:
new_reviews_data_df = reviews_data_df[['listing_id', 'comments']].copy()
new_reviews_data_df.head()

,listing_id,comments
0,2015,Mein Freund und ich hatten gute gemütliche vie...
1,2015,Jan was very friendly and welcoming host! The ...
2,2015,Un appartement tres bien situé dans un quartie...
3,2015,"It is really nice area, food, park, transport ..."
4,2015,"Buena ubicación, el departamento no está orden..."


In [7]:
new_reviews_data_df.rename(columns={'listing_id':'id'}, inplace=True)
new_reviews_data_df.head()

,id,comments
0,2015,Mein Freund und ich hatten gute gemütliche vie...
1,2015,Jan was very friendly and welcoming host! The ...
2,2015,Un appartement tres bien situé dans un quartie...
3,2015,"It is really nice area, food, park, transport ..."
4,2015,"Buena ubicación, el departamento no está orden..."


In [8]:
#new_reviews_data_df.to_csv('comments.csv')

In [9]:
review_group = new_reviews_data_df.set_index('comments').to_dict()
#review_group
# output appears liek this:
# {'id': {'MeinFreund...':2015,
#Jan was... there a lot!': 2015,... }}

In [10]:
#postgres connection
rds_connection_string = "postgres:postgres@localhost:5432/berlin_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
engine.table_names()

['listings', 'reviews']

In [18]:
# Use pandas to load csv converted DataFrame into database 
#new_listings_data_df.to_sql(name='listings', con=engine, if_exists='append', index=False)

In [13]:
 pd.read_sql_query('select * from listings', con=engine).head()

,id,name,neighbourhood,price
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,Brunnenstr. Süd,60
1,2695,Prenzlauer Berg close to Mauerpark,Prenzlauer Berg Nordwest,17
2,3176,Fabulous Flat in great Location,Prenzlauer Berg Südwest,90
3,3309,BerlinSpot Schöneberg near KaDeWe,Schöneberg-Nord,26
4,7071,BrightRoom with sunny greenview!,Helmholtzplatz,42


In [19]:
#new_reviews_data_df.to_sql(name='reviews', con=engine, if_exists='append', index=True)

In [15]:
 pd.read_sql_query('select * from reviews', con=engine).head()

,index,id,comments
0,0,2015,Mein Freund und ich hatten gute gemütliche vie...
1,1,2015,Jan was very friendly and welcoming host! The ...
2,2,2015,Un appartement tres bien situé dans un quartie...
3,3,2015,"It is really nice area, food, park, transport ..."
4,4,2015,"Buena ubicación, el departamento no está orden..."


## making new df with new csv


In [17]:
comments = "combined_comments.csv"
comments_data_df = pd.read_csv(comments)
comments_data_df.head()

,id,results
0,2015,"{""Small nice flat, perfect for one or two peop..."
1,2695,"{""Super nice and friendly. Great communication..."
2,3176,"{""Location, location, location! Britta's place..."
3,3309,"{""Nice person, nice place, nice stay."",""Jana h..."
4,7071,"{""I had a very pleasant stay at Can´s apartmen..."


In [28]:
complete_data_df = pd.concat([new_listings_data_df,comments_data_df],axis=1,join='outer')

In [29]:
complete_data_df.head()

,id,name,neighbourhood,price,id,results
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,Brunnenstr. Süd,60,2015.0,"{""Small nice flat, perfect for one or two peop..."
1,2695,Prenzlauer Berg close to Mauerpark,Prenzlauer Berg Nordwest,17,2695.0,"{""Super nice and friendly. Great communication..."
2,3176,Fabulous Flat in great Location,Prenzlauer Berg Südwest,90,3176.0,"{""Location, location, location! Britta's place..."
3,3309,BerlinSpot Schöneberg near KaDeWe,Schöneberg-Nord,26,3309.0,"{""Nice person, nice place, nice stay."",""Jana h..."
4,7071,BrightRoom with sunny greenview!,Helmholtzplatz,42,7071.0,"{""I had a very pleasant stay at Can´s apartmen..."


In [30]:
clean_complete_data_df = complete_data_df[['id', 'name', 'neighbourhood', 'price', 'results']].copy()
clean_complete_data_df.head()

,id,id,name,neighbourhood,price,results
0,2015,2015.0,Berlin-Mitte Value! Quiet courtyard/very central,Brunnenstr. Süd,60,"{""Small nice flat, perfect for one or two peop..."
1,2695,2695.0,Prenzlauer Berg close to Mauerpark,Prenzlauer Berg Nordwest,17,"{""Super nice and friendly. Great communication..."
2,3176,3176.0,Fabulous Flat in great Location,Prenzlauer Berg Südwest,90,"{""Location, location, location! Britta's place..."
3,3309,3309.0,BerlinSpot Schöneberg near KaDeWe,Schöneberg-Nord,26,"{""Nice person, nice place, nice stay."",""Jana h..."
4,7071,7071.0,BrightRoom with sunny greenview!,Helmholtzplatz,42,"{""I had a very pleasant stay at Can´s apartmen..."


In [1]:
clean_complete_data_df.to_csv('finalOutput/complete.csv')

NameError: name 'clean_complete_data_df' is not defined